In [3]:
# Packages needed

import numpy as np
import itertools
from scipy import optimize

In [ ]:
# Inputs

regularization # Parameter from the objective function
X # features' matrix
Y # target array
beta # current coefficients' array
delta_B # from your part

# Parameters to set for the line search
delta = .2
b = .5
sigma = .01
gamma = 0

In [29]:
# Bricks for the line search

def loss_function(X,Y,beta):
    y_beta_x = np.multiply(Y,X.dot(beta))
    return sum(np.log(1+np.exp(-y_beta_x)))

def penalty(beta,regularization):
    return regularization*np.linalg.norm(beta, ord=1)

def functional(X,Y,beta,regularization):
    return loss_function(X,Y,beta)+penalty(beta,regularization)

def gradient_loss(X,Y,beta):
    gradient = []
    for j in range(len(beta)):
        y_beta_x = Y*X.dot(beta)
        y_xj = Y*X[:,j]
        gradient_betaj = sum((-y_xj*np.exp(-y_beta_x))/(1+np.exp(-y_beta_x)))
        gradient.append(gradient_betaj)
    return np.array(gradient)

def second_gradient_loss(X,Y,beta,variable_j,variable_l):
    y_beta_x = Y*X.dot(beta)
    ysq_xj_xl = Y**2*(X[:,variable_j]*X[:,variable_l])
    gradient_betajl = sum((ysq_xj_xl*np.exp(-y_beta_x)*(1+np.exp(-y_beta_x)) + Y*(X[:,variable_l])*np.exp(-y_beta_x)*(-Y*(X[:,variable_j])*np.exp(-y_beta_x)))/(1+np.exp(-y_beta_x))**2)
    return gradient_betajl

def H_tilde(X,Y,beta):
    # Combination of all possible indexes
    iterator = list(itertools.product([i for i in range(len(beta))], [i for i in range(len(beta))]))
    # Initialize H_tilde matrix
    H_tilde = np.empty((len(beta),len(beta),))
    H_tilde[:] = np.nan
    for i in iterator:
        H_tilde[i[0],i[1]] = second_gradient_loss(X,Y,beta,i[0],i[1])
    return H_tilde

def D(X,Y,beta,regularization):
    return gradient_loss(X,Y,beta).dot(delta_B) + gamma*delta_B.dot(H_tilde(X,Y,beta)).dot(delta_B) + regularization*(np.linalg.norm(beta-delta_B, ord=1)-np.linalg.norm(beta, ord=1))

In [30]:
# Line search

def line_search(X,Y,beta,regularization, delta_B, sufficient_decrease, delta, b, sigma, gamma):
    if functional(X,Y,beta,regularization) - functional(X,Y,beta+delta_B,regularization) > sufficient_decrease:
        return 1
    else:
        result = optimize.minimize_scalar(lambda x: functional(X,Y,beta+x*delta_B,regularization), bounds=(delta,1), method='Bounded')
        alpha_in = result.x
        alpha = alpha_in
        count = 0
        while functional(X,Y,beta+alpha*delta_B,regularization) > functional(X,Y,beta,regularization) + alpha*sigma*D(X,Y,beta,regularization):
            count += 1
            alpha *= b**count
    return alpha